In [1]:
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import math

# Data-Processing

In [2]:
yjmob1 = 'yjmob100k-dataset1.csv.gz' # dataset under normal scenes
yjmob_df = pd.read_csv(yjmob1, compression='gzip').sort_values(by=['uid', 'd', 't'], ignore_index=True)

# Retrieve all ids
uids = yjmob_df['uid'].unique()

# Just to reduce memory space
rand_indicies = [random.randint(0, len(uids)) for _ in range(200)] # only 200 data would be used
selected_uids = [uid for uid in uids[rand_indicies]] # selected_uids = uids[:200]
# selected_uids = uids[:200]

df = yjmob_df[yjmob_df['uid'].isin(selected_uids)] 

# Time
df['combined_t'] = df['d']*47+df['t']

# Location
def spatial_token(x, y):
    return (x-1)+(y-1)*200
df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)

# Sort value
df = df.sort_values(by=['uid', 'combined_t'])

/var/folders/hx/xp23lpqx4ndfxcvp3qj_bdgr0000gn/T/ipykernel_1061/3526287213.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_t'] = df['d']*47+df['t']
/var/folders/hx/xp23lpqx4ndfxcvp3qj_bdgr0000gn/T/ipykernel_1061/3526287213.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)


# Train-Test Split 

In [3]:
# 7:3 split
train_uids, test_uids = train_test_split(selected_uids, test_size=0.30, random_state=42)

# Load training and testing data
df_train = df[df['uid'].isin(train_uids)]
df_test = df[df['uid'].isin(test_uids)]

# Batching 

In [4]:
BATCH_SIZE = 50
STEP_SIZE = 600

In [5]:
def generate_sequences(data, data_t):
    return torch.tensor(data[:STEP_SIZE]),torch.tensor(data[STEP_SIZE]),\
                torch.tensor(data_t[:STEP_SIZE]),torch.tensor(data_t[STEP_SIZE])

In [6]:
# Group data by uid
grouped_data_train = df_train[['uid', 'combined_t', 'combined_xy']].groupby('uid')
grouped_data_train = [group for _, group in df_train.groupby('uid')]
grouped_data_test = df_test[['uid', 'combined_t', 'combined_xy']].groupby('uid')
grouped_data_test = [group for _, group in df_test.groupby('uid')]

In [7]:
class TrajectoryDataset(Dataset):
    def __init__(self, grouped_data):
        self.data = grouped_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_for_uid = self.data[idx]
        inputs, labels, positions, label_positions = generate_sequences(
                                                         data_for_uid['combined_xy'].values.tolist(),
                                                         data_for_uid['combined_t'].values.tolist())
        return inputs, labels, positions, label_positions

train_dataset = TrajectoryDataset(grouped_data_train)
test_dataset = TrajectoryDataset(grouped_data_test)

In [8]:
def collate_fn(batch):
    # Unzip all batch
    inputs_batch, labels_batch, positions_batch, label_positions_batch = zip(*batch)
    
    # Pad the sequence with less length in a batch
    inputs_padded = torch.nn.utils.rnn.pad_sequence(inputs_batch, padding_value=0.0, batch_first=True)
    labels_padded = torch.tensor(np.array(labels_batch))
    positions_padded = torch.nn.utils.rnn.pad_sequence(positions_batch, padding_value=0, batch_first=True)
    label_positions_padded = torch.tensor(np.array(label_positions_batch))
    
    return inputs_padded, labels_padded, positions_padded, label_positions_padded

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [9]:
# Example

for inputs, labels, positions, label_positions in test_dataloader:
    print("Input Shape:", inputs.shape) # torch.Size([50, 600])
    print("Desired Output Shape:", labels.shape) # torch.Size([50])
    print("Positional Data Shape:", positions.shape) # torch.Size([50, 600])
    print("Corresponding Positional Data Shape: ", label_positions.shape) # torch.Size([50])
    break

Input Shape: torch.Size([50, 600])
Desired Output Shape: torch.Size([50])
Positional Data Shape: torch.Size([50, 600])
Corresponding Positional Data Shape:  torch.Size([50])


# Transformer

## Training

In [20]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, embedding_dim):
        super(PositionalEncoding, self).__init__()
        self.embedding_dim = embedding_dim

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-np.log(10000.0) / embedding_dim))
        pe = torch.zeros(max_len, 1, embedding_dim)
        pe[:, 0, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 0, 1::2] = torch.cos(position.float() * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x 
    
class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, nhead, 
                 num_encoder_layers, num_decoder_layers, dim_feedforward, max_len):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.pos_encoder = PositionalEncoding(max_len, embed_dim)
        self.transformer = nn.Transformer(embed_dim, nhead, 
                                          num_encoder_layers, num_decoder_layers, dim_feedforward)
        self.decoder = nn.Linear(embed_dim, input_dim)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src): # , src_mask
        src = self.embedding(src) * math.sqrt(self.embedding.embedding_dim)
        src = self.pos_encoder(src)
        output = self.transformer(src, src) # , src_mask
        output = self.decoder(output)
        
        # Select the output of the last timestep for each sequence in the batch
        final_output = output[:, -1, :]  # [batch_size, feature_size]
        return final_output

In [21]:
# def generate_square_subsequent_mask(sz):
#     mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#     mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#     return mask

# Model Initialization
input_dim = 200*200
embed_dim = 64 
nhead = 8
num_encoder_layers = 3
num_decoder_layers = 3
dim_feedforward = 4
max_len = 3600
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerModel(input_dim, embed_dim, nhead, 
                         num_encoder_layers, num_decoder_layers, dim_feedforward, max_len)

# Training Preparation
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mask for src
# src_mask = generate_square_subsequent_mask(STEP_SIZE).to(device)

In [26]:
%%time 

# Training
epochs = 80
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels, positions, label_positions in train_dataloader:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass: Model outputs the logits for the predicted location
        # Output shape: [batch_size, grid_size]
        output = model(inputs)

        # Label shape: [batch_size]
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch} Average Loss {average_loss:}")

Epoch 0 Average Loss 10.570607821146647
CPU times: user 31.2 s, sys: 1min 16s, total: 1min 48s
Wall time: 29.1 s


In [ ]:
# Inference

with torch.no_grad():
    for inputs, labels, positions, label_positions in test_dataloader:
        inputs = inputs.to(device)
        outputs = model(inputs) # , src_mask
        
        # Apply softmax to convert to probability distribution
        probabilities = torch.softmax(outputs, dim=-1)
        predicted_labels = probabilities.argmax(dim=-1) # 50, 600
        
        print(predicted_labels)
        print(labels)
        break